In [29]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import mapclassify
from matplotlib.patches import Patch
from pathlib import Path
from rapidfuzz import process, fuzz
from shapely.geometry import Point
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
from rasterio.mask import mask


In [30]:
# Set base project path
base_path = Path("C:/Users/juami/Dropbox/My-Research/Deforestation/data")

# Set file paths
muni_path = base_path / "Gis" / "Colombia" / "Municipios.shp"
bii_path = base_path / "BII"

export_path = base_path / "BII"

In [31]:
# Read the shapefiles and CSV files
muniShape = gpd.read_file(muni_path)

# Read the first band
bii00 = rasterio.open(bii_path / "bii-2000_v2-1-1.tif")
bii05 = rasterio.open(bii_path / "bii-2005_v2-1-1.tif")
bii10 = rasterio.open(bii_path / "bii-2010_v2-1-1.tif")
bii15 = rasterio.open(bii_path / "bii-2015_v2-1-1.tif")
bii20 = rasterio.open(bii_path / "bii-2020_v2-1-1.tif")

# Reproject shapefile to match raster CRS CRS
muniShape = muniShape.to_crs(bii00.crs)

In [32]:
# Dictionary of rasters and their corresponding years
rasters = {
    2000: bii00,
    2005: bii05,
    2010: bii10,
    2015: bii15,
    2020: bii20
}

# Loop through rasters and process
for year, raster in rasters.items():

    # Crop raster to shapefile
    cropped_data, cropped_transform = mask(raster, muniShape.geometry, crop=True)

    # Compute zonal statistics (mean within each polygon)
    stats = zonal_stats(muniShape, cropped_data[0], affine=cropped_transform, stats=["mean"], geojson_out=True)

    # Convert to GeoDataFrame and process columns
    gdf_bii = gpd.GeoDataFrame.from_features(stats)
    gdf_bii.rename(columns={"mean": "bii"}, inplace=True)
    gdf_bii.rename(columns={"IDDANE": "coddane"}, inplace=True)
    gdf_bii["year"] = year

    # Keep only relevant columns
    gdf_bii = gdf_bii[["coddane", "bii", "year"]]

    # Export to CSV
    gdf_bii.to_csv(export_path / f"muni_bii_{year}.csv", index=False)

    print(f"Exported muni_bii_{year}.csv")


Exported muni_bii_2000.csv
Exported muni_bii_2005.csv
Exported muni_bii_2010.csv
Exported muni_bii_2015.csv
Exported muni_bii_2020.csv
